In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
from numpy import arange
from sklearn.preprocessing import LabelBinarizer

In [2]:
df = pd.read_csv('master_data_pest2.csv')
del df['Unnamed: 0']
df

,Year,Nutrient nitrogen N (total) - tonnes,Nutrient phosphate P2O5 (total) - tonnes,Nutrient potash K2O (total) - tonnes,Item,harvest_area (ha),Yield(tonnes/ha),Fung & Bact – Benzimidazoles,"Fung & Bact – Diazines, morpholines",Fung & Bact – Dithiocarbamates,...,Rodenticides – Other,Seed Treat Fung – Benzimidazoles,Seed Treat Fung – Botanical products and biologicals,Seed Treat Fung – Dithiocarbamates,Seed Treat Fung – Other,"Seed Treat Fung – Triazoles, diazoles",Seed Treat Insect – Carbamates,Seed Treat Insect – Organo-phosphates,Seed Treat Insect – Other,Seed Treat Insect – Pyrethroids
0,1990,1787300,609000,954400,Apples,84400,26.3272,215.0,1047.0,2828.0,...,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1990,1787300,609000,954400,Apricots,685,2.5781,215.0,1047.0,2828.0,...,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1990,1787300,609000,954400,Asparagus,7971,2.7611,215.0,1047.0,2828.0,...,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1990,1787300,609000,954400,Barley,2612519,5.3558,215.0,1047.0,2828.0,...,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1990,1787300,609000,954400,"Beans, dry",2025,3.0736,215.0,1047.0,2828.0,...,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1569,2020,1265477,192182,446396,Sugar beet,386000,74.1402,32.0,52.0,1655.0,...,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1570,2020,1265477,192182,446396,Sunflower seed,28200,2.0567,32.0,52.0,1655.0,...,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1571,2020,1265477,192182,446396,Tomatoes,380,268.7368,32.0,52.0,1655.0,...,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1572,2020,1265477,192182,446396,Triticale,341300,5.9663,32.0,52.0,1655.0,...,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Find missing values
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
display(missing_value_df)

,column_name,percent_missing
Year,Year,0.000000
Nutrient nitrogen N (total) - tonnes,Nutrient nitrogen N (total) - tonnes,0.000000
Nutrient phosphate P2O5 (total) - tonnes,Nutrient phosphate P2O5 (total) - tonnes,0.000000
Nutrient potash K2O (total) - tonnes,Nutrient potash K2O (total) - tonnes,0.000000
Item,Item,0.000000
harvest_area (ha),harvest_area (ha),0.000000
Yield(tonnes/ha),Yield(tonnes/ha),0.000000
Fung & Bact – Benzimidazoles,Fung & Bact – Benzimidazoles,0.000000
"Fung & Bact – Diazines, morpholines","Fung & Bact – Diazines, morpholines",0.000000
Fung & Bact – Dithiocarbamates,Fung & Bact – Dithiocarbamates,0.000000


In [4]:
df_del = missing_value_df[missing_value_df['percent_missing'] > 30]
cols_del = list(df_del['column_name'])
cols_del


['Herbicides – Uracil',
 'Insecticides – Chlorinated Hydrocarbons',
 'Seed Treat Fung – Benzimidazoles',
 'Seed Treat Fung – Botanical products and biologicals',
 'Seed Treat Fung – Dithiocarbamates',
 'Seed Treat Fung – Other',
 'Seed Treat Fung – Triazoles, diazoles',
 'Seed Treat Insect – Carbamates',
 'Seed Treat Insect – Organo-phosphates',
 'Seed Treat Insect – Other',
 'Seed Treat Insect – Pyrethroids']

In [6]:
#for i in cols_del:
#    del df[i]

new_df = df.copy()
new_df

,Year,Nutrient nitrogen N (total) - tonnes,Nutrient phosphate P2O5 (total) - tonnes,Nutrient potash K2O (total) - tonnes,Item,harvest_area (ha),Yield(tonnes/ha),Fung & Bact – Benzimidazoles,"Fung & Bact – Diazines, morpholines",Fung & Bact – Dithiocarbamates,...,Herbicides – Urea derivates,Insecticides – Botanical products and biologicals,Insecticides – Carbamates,Insecticides – Organo-phosphates,Insecticides – Other,Insecticides – Pyrethroids,Mineral Oils,Plant Growth Regulators,Rodenticides – Anti-coagulants,Rodenticides – Other
0,1990,1787300,609000,954400,Apples,84400,26.3272,215.0,1047.0,2828.0,...,3825.0,1.0,316.0,891.0,396.0,113.0,642.0,1910.0,5.0,8.0
1,1990,1787300,609000,954400,Apricots,685,2.5781,215.0,1047.0,2828.0,...,3825.0,1.0,316.0,891.0,396.0,113.0,642.0,1910.0,5.0,8.0
2,1990,1787300,609000,954400,Asparagus,7971,2.7611,215.0,1047.0,2828.0,...,3825.0,1.0,316.0,891.0,396.0,113.0,642.0,1910.0,5.0,8.0
3,1990,1787300,609000,954400,Barley,2612519,5.3558,215.0,1047.0,2828.0,...,3825.0,1.0,316.0,891.0,396.0,113.0,642.0,1910.0,5.0,8.0
4,1990,1787300,609000,954400,"Beans, dry",2025,3.0736,215.0,1047.0,2828.0,...,3825.0,1.0,316.0,891.0,396.0,113.0,642.0,1910.0,5.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1569,2020,1265477,192182,446396,Sugar beet,386000,74.1402,32.0,52.0,1655.0,...,665.0,20.0,86.0,0.0,20739.0,121.0,359.0,2233.0,NaN,16.0
1570,2020,1265477,192182,446396,Sunflower seed,28200,2.0567,32.0,52.0,1655.0,...,665.0,20.0,86.0,0.0,20739.0,121.0,359.0,2233.0,NaN,16.0
1571,2020,1265477,192182,446396,Tomatoes,380,268.7368,32.0,52.0,1655.0,...,665.0,20.0,86.0,0.0,20739.0,121.0,359.0,2233.0,NaN,16.0
1572,2020,1265477,192182,446396,Triticale,341300,5.9663,32.0,52.0,1655.0,...,665.0,20.0,86.0,0.0,20739.0,121.0,359.0,2233.0,NaN,16.0


In [7]:
percent_missing1 = new_df.isnull().sum() * 100 / len(new_df)
missing_value_df1 = pd.DataFrame({'column_names': new_df.columns,
                                 'percentage_missing': percent_missing1})
display(missing_value_df1)

,column_names,percentage_missing
Year,Year,0.000000
Nutrient nitrogen N (total) - tonnes,Nutrient nitrogen N (total) - tonnes,0.000000
Nutrient phosphate P2O5 (total) - tonnes,Nutrient phosphate P2O5 (total) - tonnes,0.000000
Nutrient potash K2O (total) - tonnes,Nutrient potash K2O (total) - tonnes,0.000000
Item,Item,0.000000
harvest_area (ha),harvest_area (ha),0.000000
Yield(tonnes/ha),Yield(tonnes/ha),0.000000
Fung & Bact – Benzimidazoles,Fung & Bact – Benzimidazoles,0.000000
"Fung & Bact – Diazines, morpholines","Fung & Bact – Diazines, morpholines",0.000000
Fung & Bact – Dithiocarbamates,Fung & Bact – Dithiocarbamates,0.000000


In [8]:
df_imp = missing_value_df1[missing_value_df1['percentage_missing'] > 0]
cols_imp = list(df_imp['column_names'])
cols_imp

['Herbicides – Bipiridils',
 'Herbicides – Sulfonyl ureas',
 'Rodenticides – Anti-coagulants']

In [9]:
imp_df = new_df[cols_imp]
imp_df

,Herbicides – Bipiridils,Herbicides – Sulfonyl ureas,Rodenticides – Anti-coagulants
0,NaN,NaN,5.0
1,NaN,NaN,5.0
2,NaN,NaN,5.0
3,NaN,NaN,5.0
4,NaN,NaN,5.0
...,...,...,...
1569,0.0,134.0,NaN
1570,0.0,134.0,NaN
1571,0.0,134.0,NaN
1572,0.0,134.0,NaN


In [10]:
imp_df['Rodenticides – Anti-coagulants'].value_counts()

0.10    309
0.00    201
1.00    197
0.41    108
0.04    108
0.01    108
0.03     54
6.00     52
5.00     50
4.00     50
0.30     50
0.20     50
Name: Rodenticides – Anti-coagulants, dtype: int64

In [11]:
imputer = KNNImputer(n_neighbors=30)
knni_df = pd.DataFrame(imputer.fit_transform(new_df[cols_imp]))
#display(df)
display(knni_df)

,0,1,2
0,139.766667,41.6,5.0
1,139.766667,41.6,5.0
2,139.766667,41.6,5.0
3,139.766667,41.6,5.0
4,139.766667,41.6,5.0
...,...,...,...
1569,0.000000,134.0,0.1
1570,0.000000,134.0,0.1
1571,0.000000,134.0,0.1
1572,0.000000,134.0,0.1


In [12]:
knni_df = knni_df.rename(columns={0: 'Herbicides – Bipiridils', 1: 'Herbicides – Sulfonyl ureas', 2: 'Rodenticides – Anti-coagulants'})
knni_df

,Herbicides – Bipiridils,Herbicides – Sulfonyl ureas,Rodenticides – Anti-coagulants
0,139.766667,41.6,5.0
1,139.766667,41.6,5.0
2,139.766667,41.6,5.0
3,139.766667,41.6,5.0
4,139.766667,41.6,5.0
...,...,...,...
1569,0.000000,134.0,0.1
1570,0.000000,134.0,0.1
1571,0.000000,134.0,0.1
1572,0.000000,134.0,0.1


In [13]:
del new_df['Herbicides – Bipiridils']
del new_df['Herbicides – Sulfonyl ureas']
del new_df['Rodenticides – Anti-coagulants']
del new_df['harvest_area (ha)']

In [14]:
final_df = new_df.merge(knni_df,left_index= True, right_index=True)
final_df

,Year,Nutrient nitrogen N (total) - tonnes,Nutrient phosphate P2O5 (total) - tonnes,Nutrient potash K2O (total) - tonnes,Item,Yield(tonnes/ha),Fung & Bact – Benzimidazoles,"Fung & Bact – Diazines, morpholines",Fung & Bact – Dithiocarbamates,Fung & Bact – Inorganics,...,Insecticides – Carbamates,Insecticides – Organo-phosphates,Insecticides – Other,Insecticides – Pyrethroids,Mineral Oils,Plant Growth Regulators,Rodenticides – Other,Herbicides – Bipiridils,Herbicides – Sulfonyl ureas,Rodenticides – Anti-coagulants
0,1990,1787300,609000,954400,Apples,26.3272,215.0,1047.0,2828.0,2466.0,...,316.0,891.0,396.0,113.0,642.0,1910.0,8.0,139.766667,41.6,5.0
1,1990,1787300,609000,954400,Apricots,2.5781,215.0,1047.0,2828.0,2466.0,...,316.0,891.0,396.0,113.0,642.0,1910.0,8.0,139.766667,41.6,5.0
2,1990,1787300,609000,954400,Asparagus,2.7611,215.0,1047.0,2828.0,2466.0,...,316.0,891.0,396.0,113.0,642.0,1910.0,8.0,139.766667,41.6,5.0
3,1990,1787300,609000,954400,Barley,5.3558,215.0,1047.0,2828.0,2466.0,...,316.0,891.0,396.0,113.0,642.0,1910.0,8.0,139.766667,41.6,5.0
4,1990,1787300,609000,954400,"Beans, dry",3.0736,215.0,1047.0,2828.0,2466.0,...,316.0,891.0,396.0,113.0,642.0,1910.0,8.0,139.766667,41.6,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1569,2020,1265477,192182,446396,Sugar beet,74.1402,32.0,52.0,1655.0,2851.0,...,86.0,0.0,20739.0,121.0,359.0,2233.0,16.0,0.000000,134.0,0.1
1570,2020,1265477,192182,446396,Sunflower seed,2.0567,32.0,52.0,1655.0,2851.0,...,86.0,0.0,20739.0,121.0,359.0,2233.0,16.0,0.000000,134.0,0.1
1571,2020,1265477,192182,446396,Tomatoes,268.7368,32.0,52.0,1655.0,2851.0,...,86.0,0.0,20739.0,121.0,359.0,2233.0,16.0,0.000000,134.0,0.1
1572,2020,1265477,192182,446396,Triticale,5.9663,32.0,52.0,1655.0,2851.0,...,86.0,0.0,20739.0,121.0,359.0,2233.0,16.0,0.000000,134.0,0.1


In [15]:
final_df.to_csv("master_data_processed.csv")